
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [20]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

In [40]:
path = '../data/AK_Juneau_0.json'
with open(path, 'r') as file:
    file1 = json.load(file)
data = pd.json_normalize(file1['data']['results'])
# load one file first to see what type of data you're dealing with and what attributes it has

At this point, ensure that you have all sales in a dataframe.
- Is each cell one value, or do some cells have lists?
- Maybe the "tags" will help create some features.
- What are the data types of each column?
- Some sales may not actually include the sale price.  These rows should be dropped.
- Some sales don't include the property type.
- There are a lot of None values.  Should these be dropped or replaced with something?

In [49]:
import glob

json_directory = '../data/'

json_file_paths = glob.glob(os.path.join(json_directory, '*.json'))

for path in json_file_paths:
    with open(path, 'r') as file:
        jsonfile = json.load(file)
    newdata = pd.json_normalize(jsonfile['data']['results'])
    data = pd.concat([newdata, data])

data
# load and concatenate data here
# drop or replace values as necessary

,last_update_date,tags,permalink,status,list_date,open_houses,branding,list_price,property_id,photos,...,location.county.fips_code,location.county.name,primary_photo,source,products,other_listings,location.address.coordinate,location.county,community.advertisers,community.description.name
0,2023-12-30T18:44:03Z,"[central_air, community_golf, community_spa_or...",1938-Golf-Stream-Dr_Indianapolis_IN_46229_M359...,sold,2023-11-17T01:45:13.000000Z,None,"[{'name': 'FC Tucker Company Zionsville', 'pho...",269000.0,3598513543,"[{'tags': [{'label': 'garage', 'probability': ...",...,18097,Marion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-03T19:12:44Z,"[central_air, dining_room, fireplace, forced_a...",5620-Noble-Dr_Indianapolis_IN_46234_M36433-44288,sold,2023-09-15T21:15:15.000000Z,None,"[{'name': 'Exp Realty, Llc', 'photo': None, 't...",310000.0,3643344288,"[{'tags': [{'label': 'house_view', 'probabilit...",...,18063,Hendricks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-12-30T18:37:09Z,"[central_air, community_security_features, din...",1019-Schmitt-Rd_Indianapolis_IN_46239_M36218-9...,sold,2023-10-20T14:30:29.000000Z,None,"[{'name': 'FC Tucker Company Zionsville', 'pho...",385000.0,3621895033,"[{'tags': [{'label': 'house_view', 'probabilit...",...,18097,Marion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-12-22T23:58:57Z,"[central_air, community_spa_or_hot_tub, dining...",7192-Bay-View-Dr_Indianapolis_IN_46214_M34219-...,sold,2023-10-26T01:32:35.000000Z,None,"[{'name': 'Carrie Gruel & Associates, REALTORS...",169900.0,3421931060,"[{'tags': [{'label': 'garage', 'probability': ...",...,18097,Marion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-12-20T19:45:03Z,"[city_view, community_park, community_spa_or_h...",10928-Spring-Green-Dr_Indianapolis_IN_46229_M3...,sold,2023-11-12T18:50:36.000000Z,None,"[{'name': 'Next Home Elite Real Estate', 'phot...",310000.0,3006648843,"[{'tags': [{'label': 'garage', 'probability': ...",...,18097,Marion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,None,None,17850-Point-Stephens-Rd_Juneau_AK_99801_M98793...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9879331943,None,...,None,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,None,9951-Stephen-Richards-Memorial-Dr_Juneau_AK_99...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9521639574,None,...,None,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-08-04T22:49:14Z,"[carport, hill_or_mountain_view, ocean_view, s...",11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,[{'name': 'Platinum Keller Williams Realty Ala...,415000.0,7412309719,None,...,None,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023-08-04T21:31:44Z,"[city_view, community_outdoor_space, community...",808-5th-St_Juneau_AK_99801_M76604-90777,sold,2023-08-21T21:01:22.000000Z,None,"[{'name': 'Southeast Alaska Real Estate', 'pho...",835000.0,7660490777,None,...,None,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables here
# tags will have to be done manually

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- If you replace cities or states with numerical values, make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [ ]:
# perform train test split here
# do something with state and city
# drop any other not needed columns

**STRETCH**

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized